## Nuotraukų atsisiuntimas iš COCO duomenų bazės

In [1]:
from pycocotools.coco import COCO
import os
import requests
from PIL import Image
import numpy as np
from collections import defaultdict

### Pagalbinės funkcijos, kurios sukuria kaukes iš duomenų esančių 'annotations' aplanke

Kaukės yra nuspalvinamos 3 spalvomis: lėktuvai raudona, automobiliai žalia ir žmonės mėlyna spalvomis.

In [2]:
category_colors = {
    'airplane': (255, 0, 0),  # Red
    'car': (0, 255, 0),       # Green
    'person': (0, 0, 255)     # Blue
}

def download_image(img_url, path):
    try:
        response = requests.get(img_url, stream=True)
        if response.status_code == 200:
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            print(f"Failed to download image {img_url}")
    except requests.RequestException as e:
        print(f"Error downloading {img_url}: {str(e)}")

def generate_mask(coco, img_id, img_details, mask_path, category_ids):
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=category_ids, iscrowd=None)
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((img_details['height'], img_details['width'], 3), dtype=np.uint8)

    # Sort annotations by area, larger first to allow smaller items not to be hidden entirely
    for ann in sorted(anns, key=lambda x: x['area'], reverse=True):
        category_name = coco.loadCats(ann['category_id'])[0]['name']
        binary_mask = coco.annToMask(ann)
        if category_name in category_colors:
            category_color = np.array(category_colors[category_name], dtype=np.uint8)
            color_mask = np.stack([binary_mask] * 3, axis=-1) * category_color
            # Apply color only where no color has been applied yet
            mask = np.where(mask.any(axis=2, keepdims=True), mask, color_mask)

    Image.fromarray(mask).save(mask_path)


def setup_directories(base_dir, sub_dirs):
    for sub_dir in sub_dirs:
        os.makedirs(os.path.join(base_dir, sub_dir, 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_dir, sub_dir, 'masks'), exist_ok=True)

In [3]:
classes = ['airplane', 'car', 'person']
max_samples_training_test = 1700 # per class
max_samples_validation = 300 # per class
categories = {'airplane': (255, 0, 0), 'car': (0, 255, 0), 'person': (0, 0, 255)}
dataset_dir = 'dataset'
annotations_dir = 'dataset/annotations'

### Pagrindinės funkcijos, atsiunčiančios nuotraukas iš COCO duomenų bazės

In [4]:
setup_directories(dataset_dir, ['train', 'val'])
train_sample_count = defaultdict(int)
val_sample_count = defaultdict(int)

def process_images_and_masks(coco, dataset_dir, phase, sample_count, max_samples):
    category_ids = coco.getCatIds(catNms=categories.keys())
    img_ids = coco.getImgIds()

    for img_id in img_ids:
        img_info = coco.loadImgs(img_id)[0]
        annotations = coco.loadAnns(coco.getAnnIds(imgIds=[img_id]))
        categories_in_image = {ann['category_id'] for ann in annotations if ann['category_id'] in category_ids}

        if not categories_in_image:
            continue

        img_path = os.path.join(dataset_dir, phase, 'images', img_info['file_name'])
        mask_path = os.path.join(dataset_dir, phase, 'masks', img_info['file_name'].replace('.jpg', '_mask.png'))

        if os.path.exists(img_path) and os.path.exists(mask_path):
            continue

        relevant_categories = [cat for cat in categories_in_image if sample_count[coco.loadCats(cat)[0]['name']] < max_samples]

        if not relevant_categories:
            continue

        download_image(img_info['coco_url'], img_path)
        generate_mask(coco, img_id, img_info, mask_path, relevant_categories)

        for cat_id in relevant_categories:
            category_name = coco.loadCats(cat_id)[0]['name']
            sample_count[category_name] += 1

        if all(count >= max_samples for count in sample_count.values()):
            break

for phase in ['train', 'val']:
    coco_annotation = f"{annotations_dir}/instances_{phase}2017.json"
    coco = COCO(coco_annotation)
    sample_count = train_sample_count if phase == 'train' else val_sample_count
    process_images_and_masks(coco, dataset_dir, phase, sample_count, max_samples_training_test if phase == 'train' else max_samples_validation)

    print(f"Finished processing {phase}. Counts: {dict(sample_count)}")
print("Finished processing all images.")

loading annotations into memory...
Done (t=40.65s)
creating index...
index created!
Finished processing train. Counts: {'person': 1700, 'car': 1700, 'airplane': 1700}
loading annotations into memory...
Done (t=36.77s)
creating index...
index created!
Finished processing val. Counts: {'person': 300, 'car': 300, 'airplane': 97}
Finished processing all images.
